  
# Python User Group Grenoble

## Docker + Python, Python + Docker

### Michael Bright, 3 juillet 2017, RMLL, St-Etienne

<a name="TOP" />

Agenda

- <a href="#PYTHON_APPS"> * </a> Running Python apps under Docker

- <a href="#API"> * </a>The Docker API

- <a href="#MACHINE"> * </a>Docker-machine from Python

- <a href="#DOCKERPY"> * </a>docker-py: Controlling Docker from Python

- <a href="#OTHERPY"> * </a>Other Modules

- <a href="#ANSIBLE"> * </a>Ansible Container

<a name="PYTHON_APPS" />
# Running Python apps under Docker
<a href="#TOP"> TOP </a>

Of course we can run any Python app under Docker.

This can be a very useful alternative to virtualenv
- we can have complete filesystem isolation allowing to have complex dependancies beyond Python itself
- we can easily launch multiple instances of a Python process in isolation

In [ ]:
!docker stop $(docker ps -qa)

## Flask

In [1]:
cd ~/z/bin/DEMOS_TUTORIALS/2017-May-PyconUS/docker-py/flask-app

/home/mjb/Dropbox/z/bin/DEMOS_TUTORIALS/2017-May-PyconUS/docker-py/flask-app


In [2]:
!cat flask_app.py

#!/usr/bin/env python

from flask import Flask
app = Flask(__name__)

''' Create the default '/' route '''
@app.route('/')
def root_handler():
    return 'Hello World from Flask - /'

''' Create an alternative '/test' route '''
@app.route('/test')
def test_handler():
    return 'Hello World from Flask - /test'

if __name__ == '__main__':
    app.run(debug=True,host='0.0.0.0')



In [3]:
cat Dockerfile

FROM ubuntu:latest

LABEL MAINTAINER Mike Bright "dockerfile@mjbright.net"

# Install pip:
RUN apt-get update -y && \
    apt-get install -y python-pip python-dev build-essential

WORKDIR /app

# Copy in requirements before the source files: as likely to be updated less frequently
COPY    ./requirements.txt /app/

# Install the Flask module:
RUN pip install -r requirements.txt

# Copy in the applicaton script:
COPY    ./flask_app.py     /app/

# Set the default command:
ENTRYPOINT ["python"]
CMD        ["flask_app.py"]



In [4]:
!docker build -t mjbright/flaskapp .

Sending build context to Docker daemon  4.608kB
Step 1/9 : FROM ubuntu:latest
 ---> d355ed3537e9
Step 2/9 : LABEL MAINTAINER Mike Bright "dockerfile@mjbright.net"
 ---> Using cache
 ---> 6ee2e95b312a
Step 3/9 : RUN apt-get update -y &&     apt-get install -y python-pip python-dev build-essential
 ---> Using cache
 ---> 51c4db36528c
Step 4/9 : WORKDIR /app
 ---> Using cache
 ---> 859a5b98bd66
Step 5/9 : COPY ./requirements.txt /app/
 ---> Using cache
 ---> 7a681c19a582
Step 6/9 : RUN pip install -r requirements.txt
 ---> Using cache
 ---> e0b779c11c92
Step 7/9 : COPY ./flask_app.py /app/
 ---> Using cache
 ---> fe8991ce30d2
Step 8/9 : ENTRYPOINT python
 ---> Using cache
 ---> 12cdb6f03a53
Step 9/9 : CMD flask_app.py
 ---> Using cache
 ---> 2d4ffd4a9bbc
Successfully built 2d4ffd4a9bbc
Successfully tagged mjbright/flaskapp:latest


In [5]:
!docker run -d -p 5000:5000 mjbright/flaskapp

ecf09b28660f8269c8a436f8de2e612b307a4a507c072e10db65a56954944543


In [6]:
!wget -q -O - http://localhost:5000/

Hello World from Flask - /

In [7]:
!wget -q -O - http://localhost:5000/test

Hello World from Flask - /test

## Cherrypy

In [8]:
cd ~/z/bin/DEMOS_TUTORIALS/2017-May-PyconUS/docker-py/cherrypy-app

/home/mjb/Dropbox/z/bin/DEMOS_TUTORIALS/2017-May-PyconUS/docker-py/cherrypy-app


In [9]:
!cat cherrypy_app.py

#!/usr/bin/env python

import cherrypy

class Root:

    ''' Create the default '/' route '''
    @cherrypy.expose
    def index(self):
        return 'Hello World from CherryPy - /'

    ''' Create an alternative '/test' route '''
    @cherrypy.expose
    def test(self):
        return 'Hello World from CherryPy - /test'

if __name__ == '__main__':
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.quickstart(Root())


In [10]:
!cat Dockerfile

FROM ubuntu:latest

LABEL MAINTAINER Mike Bright "dockerfile@mjbright.net"

# Install pip:
RUN apt-get update -y && \
    apt-get install -y python-pip python-dev build-essential

WORKDIR /app

# Copy in requirements before the source files: as likely to be updated less frequently
COPY    ./requirements.txt /app/

# Install the CherryPy module:
RUN pip install -r requirements.txt

# Copy in the applicaton script:
COPY    ./cherrypy_app.py     /app/

# Set the default command:
ENTRYPOINT ["python"]
CMD        ["cherrypy_app.py"]


In [11]:
!docker build -t mjbright/cherrpyapp .

Sending build context to Docker daemon  4.608kB
Step 1/9 : FROM ubuntu:latest
 ---> d355ed3537e9
Step 2/9 : LABEL MAINTAINER Mike Bright "dockerfile@mjbright.net"
 ---> Using cache
 ---> 6ee2e95b312a
Step 3/9 : RUN apt-get update -y &&     apt-get install -y python-pip python-dev build-essential
 ---> Using cache
 ---> 51c4db36528c
Step 4/9 : WORKDIR /app
 ---> Using cache
 ---> 859a5b98bd66
Step 5/9 : COPY ./requirements.txt /app/
 ---> Using cache
 ---> 977309ea7e05
Step 6/9 : RUN pip install -r requirements.txt
 ---> Using cache
 ---> 1d57541b9b7a
Step 7/9 : COPY ./cherrypy_app.py /app/
 ---> Using cache
 ---> e6602881d27b
Step 8/9 : ENTRYPOINT python
 ---> Using cache
 ---> 55f831e90de2
Step 9/9 : CMD cherrypy_app.py
 ---> Using cache
 ---> eb2c01b4f1fb
Successfully built eb2c01b4f1fb
Successfully tagged mjbright/cherrpyapp:latest


In [12]:
!docker run -d -p 9191:8080 mjbright/cherrpyapp

b0991896976d694c5a97aa1f32a10b5af7943572d2b4b66d69385b58c712875d


In [13]:
!wget -q -O - http://localhost:9191/

Hello World from CherryPy - /

In [15]:
!docker ps

CONTAINER ID        IMAGE                 COMMAND                  CREATED             STATUS              PORTS                    NAMES
b0991896976d        mjbright/cherrpyapp   "python cherrypy_a..."   45 seconds ago      Up 42 seconds       0.0.0.0:9191->8080/tcp   quizzical_yonath
ecf09b28660f        mjbright/flaskapp     "python flask_app.py"    4 minutes ago       Up 4 minutes        0.0.0.0:5000->5000/tcp   cocky_cori


In [14]:
!wget -q -O - http://localhost:9191/test

Hello World from CherryPy - /test

<a name="API" />
# Docker API
<a href="#TOP"> TOP </a>

Everything starts with the API whether that be
- The docker cli
- The docker-py module or other language bindings ...

So let's look at that first.

On my machine, the Docker daemon is listening on port *localhost:2475*

(**Note**: that's unusual)

In [16]:
cd ~/z/bin/DEMOS_TUTORIALS/2017-May-PyconUS/docker-py/

/home/mjb/Dropbox/z/bin/DEMOS_TUTORIALS/2017-May-PyconUS/docker-py


In [17]:
!ps -fade | grep dockerd | grep -v grep

root     27835 27833  1 Jun30 pts/16   01:08:09 /home/mjb/DOCKER/docker-17.06.0-ce/dockerd --bridge=maindocker0 --exec-root=/home/mjb/var/run/maindocker/maindocker0.exec --graph=/home/mjb/var/run/maindocker/maindocker0.graph --host unix:///home/mjb/var/run/maindocker/maindocker0.sock --host tcp://127.0.0.1:2475 --pidfile=/home/mjb/var/run/maindocker/maindocker0.pid


# Docker API - monitoring the cli client

**Note**: My Docker daemon is listening on *localhost:2475* without encryption for  experimental purposes - so we can examine  API traffic

I'm running a Python script to monitor  *localhost:2475* and save request/responses to a file.

Let's do some cli operations, and observe what goes out over the API.

In [18]:
!docker version

Client:
 Version:      17.06.0-ce
 API version:  1.30
 Go version:   go1.8.3
 Git commit:   02c1d87
 Built:        Fri Jun 23 21:15:15 2017
 OS/Arch:      linux/amd64

Server:
 Version:      17.06.0-ce
 API version:  1.30 (minimum version 1.12)
 Go version:   go1.8.3
 Git commit:   02c1d87
 Built:        Fri Jun 23 21:51:55 2017
 OS/Arch:      linux/amd64
 Experimental: false


Let's look at the request which was made to the Docker daemon

In [19]:
!cat /tmp/dump.last.req

[2017-07-03 15:32:55] REQUEST <<GET /v1.30/version HTTP/1.1
    Host: 127.0.0.1:2475
    User-Agent: Docker-Client/17.06.0-ce (linux)
    Accept-Encoding: gzip>>




Let's look at the response which we got back from the Docker daemon

In [20]:
!cat /tmp/dump.last.rsp

[2017-07-03 15:32:55] RESPONSE <<HTTP/1.1 200 OK
    Api-Version: 1.30
    Content-Type: application/json
    Docker-Experimental: false
    Ostype: linux
    Server: Docker/17.06.0-ce (linux)
    Date: Mon, 03 Jul 2017 13:32:55 GMT
    Content-Length: 231>>

RESPONSE=<<{
    "ApiVersion": "1.30", 
    "Arch": "amd64", 
    "BuildTime": "2017-06-23T21:51:55.152028673+00:00", 
    "GitCommit": "02c1d87", 
    "GoVersion": "go1.8.3", 
    "KernelVersion": "4.11.6-101.fc24.x86_64", 
    "MinAPIVersion": "1.12", 
    "Os": "linux", 
    "Version": "17.06.0-ce"
}>>


and we can compare this to the docker cli to see how it treats the information that it receives from the API.  Obviously this corresponds to the "Server:" information which comes via the API, not the "Client:" information which the client holds already

In [ ]:
!docker version

# Docker API - monitoring 'docker ps'

In [21]:
!docker container ps

CONTAINER ID        IMAGE                 COMMAND                  CREATED             STATUS              PORTS                    NAMES
b0991896976d        mjbright/cherrpyapp   "python cherrypy_a..."   10 minutes ago      Up 10 minutes       0.0.0.0:9191->8080/tcp   quizzical_yonath
ecf09b28660f        mjbright/flaskapp     "python flask_app.py"    13 minutes ago      Up 13 minutes       0.0.0.0:5000->5000/tcp   cocky_cori


Let's look at the request which was made to the Docker daemon

In [22]:
!cat /tmp/dump.last.req

[2017-07-03 15:38:25] REQUEST <<GET /v1.30/containers/json HTTP/1.1
    Host: 127.0.0.1:2475
    User-Agent: Docker-Client/17.06.0-ce (linux)
    Accept-Encoding: gzip>>




and the request we got back

In [23]:
!cat /tmp/dump.last.rsp

[2017-07-03 15:38:25] RESPONSE <<HTTP/1.1 200 OK
    Api-Version: 1.30
    Content-Type: application/json
    Docker-Experimental: false
    Ostype: linux
    Server: Docker/17.06.0-ce (linux)
    Date: Mon, 03 Jul 2017 13:38:25 GMT
    Content-Length: 1857>>

RESPONSE=<<[
{"Id":"b0991896976d694c5a97aa1f32a10b5af7943572d2b4b66d69385b58c712875d",
    "Names":[
    "/quizzical_yonath"]
      ,
    "Image":"mjbright/cherrpyapp",
    "ImageID":"sha256:eb2c01b4f1fb83799ec7cf5666668baf07be4950653e9aea5eafcff6a55ba2d8",
    "Command":"python.cherrypy_app.py",
    "Created":1499088488,
    "Ports":[
    {"IP":"0.0.0.0",
        "PrivatePort":8080,
        "PublicPort":9191,
        "Type":"tcp"}
        ],
    "Labels":{
    "MAINTAINER":"Mike.Bright.dockerfile@mjbright.net"}
      ,
    "State":"running",
    "Status":"Up.10.minutes",
    "HostConfig":{
    "NetworkMode":"default"}
      ,
    "NetworkSettings":{
    "Networks":{
      "bridge":{
        "IPAMConfig":null,
          "Links":n

Let's try that query ourselves using wget:


In [25]:
!wget -q -O - http://localhost:2475/v1.30/containers/json | jq .

[
  {
    "Id": "b0991896976d694c5a97aa1f32a10b5af7943572d2b4b66d69385b58c712875d",
    "Names": [
      "/quizzical_yonath"
    ],
    "Image": "mjbright/cherrpyapp",
    "ImageID": "sha256:eb2c01b4f1fb83799ec7cf5666668baf07be4950653e9aea5eafcff6a55ba2d8",
    "Command": "python cherrypy_app.py",
    "Created": 1499088488,
    "Ports": [
      {
        "IP": "0.0.0.0",
        "PrivatePort": 8080,
        "PublicPort": 9191,
        "Type": "tcp"
      }
    ],
    "Labels": {
      "MAINTAINER": "Mike Bright dockerfile@mjbright.net"
    },
    "State": "running",
    "Status": "Up 12 minutes",
    "HostConfig": {
      "NetworkMode": "default"
    },
    "NetworkSettings": {
      "Networks": {
        "bridge": {
          "IPAMConfig": null,
          "Links": null,
          "Aliases": null,
          "NetworkID": "151794f82880e59768799c567714ca3e8a15dd0f1c6d1957ded20a1cc67eb364",
          "EndpointID": "251540abf2297c64a9cbabad4601495011bd77149bb0e78b14ac8f18ad00a3eb",
        

# Docker API - monitoring 'docker ps -a'

Now let's specify the **-a** option to list all containers, not just running ones

In [26]:
!docker ps -a

CONTAINER ID        IMAGE                 COMMAND                  CREATED             STATUS              PORTS                    NAMES
b0991896976d        mjbright/cherrpyapp   "python cherrypy_a..."   13 minutes ago      Up 13 minutes       0.0.0.0:9191->8080/tcp   quizzical_yonath
ecf09b28660f        mjbright/flaskapp     "python flask_app.py"    16 minutes ago      Up 16 minutes       0.0.0.0:5000->5000/tcp   cocky_cori


Let's look at the request which was made to the Docker daemon

Note that using the '-a' option has modified the request url to append '?all=1':

In [27]:
!cat /tmp/dump.last.req

[2017-07-03 15:41:13] REQUEST <<GET /v1.30/containers/json?all=1 HTTP/1.1
    Host: 127.0.0.1:2475
    User-Agent: Docker-Client/17.06.0-ce (linux)
    Accept-Encoding: gzip>>




In [ ]:
!cat /tmp/dump.last.rsp

In [ ]:
!which docker

In [28]:
!docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://cloud.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/engine/userguide/



In [32]:
!docker ps -a 

CONTAINER ID        IMAGE                 COMMAND                  CREATED              STATUS                          PORTS                    NAMES
58c1a1d09ee0        hello-world           "/hello"                 About a minute ago   Exited (0) About a minute ago                            amazing_ardinghelli
b0991896976d        mjbright/cherrpyapp   "python cherrypy_a..."   16 minutes ago       Up 16 minutes                   0.0.0.0:9191->8080/tcp   quizzical_yonath
ecf09b28660f        mjbright/flaskapp     "python flask_app.py"    19 minutes ago       Up 19 minutes                   0.0.0.0:5000->5000/tcp   cocky_cori


In [29]:
!cat /tmp/dump.last.req

[2017-07-03 15:42:40] REQUEST <<POST /v1.30/containers/58c1a1d09ee00477d56cf2ff80aa627fa7ab0d64e7a8af3146ac12f44bb2c018/start HTTP/1.1
    Host: 127.0.0.1:2475
    User-Agent: Docker-Client/17.06.0-ce (linux)
    Content-Length: 0
    Content-Type: text/plain
    Accept-Encoding: gzip>>




In [31]:
!cat /tmp/dump.last.rsp

# Docker API - monitoring 'docker images'

In [35]:
!docker images -a

REPOSITORY                         TAG                 IMAGE ID            CREATED             SIZE
mjbright/cherrpyapp                latest              eb2c01b4f1fb        22 hours ago        429MB
<none>                             <none>              55f831e90de2        22 hours ago        429MB
<none>                             <none>              e6602881d27b        22 hours ago        429MB
<none>                             <none>              1d57541b9b7a        22 hours ago        429MB
<none>                             <none>              977309ea7e05        22 hours ago        424MB
mjbright/flaskapp                  latest              2d4ffd4a9bbc        22 hours ago        430MB
<none>                             <none>              12cdb6f03a53        22 hours ago        430MB
<none>                             <none>              fe8991ce30d2        22 hours ago        430MB
<none>                             <none>              e0b779c11c92        22 hours ago     

Let's look at the request which was made to the Docker daemon

Note that we use /images rather than /containers now.

In fact just as the *new* docker cli commands operate on objects, those commands are more aligned with the API way of doing things

In [36]:
!cat /tmp/dump.last.req

[2017-07-03 15:45:55] REQUEST <<GET /v1.30/images/json?all=1 HTTP/1.1
    Host: 127.0.0.1:2475
    User-Agent: Docker-Client/17.06.0-ce (linux)
    Accept-Encoding: gzip>>




Let's look at the response which we got back from the Docker daemon

In [37]:
!cat /tmp/dump.last.rsp

# Docker API - direct access

Let's double-check by performing some requests directly from the command-line


Note that 'docker search' actually queries the 'docker daemon' which then queries your registry (Docker hub by default)

In [38]:
!curl -s -XGET http://localhost:2475/images/search?term=mjbright/docker-demo | jq . 

[
  {
    "star_count": 0,
    "is_official": false,
    "name": "mjbright/docker-demo",
    "is_automated": false,
    "description": ""
  }
]


In [39]:
cat /tmp/dump.last.req

[2017-07-03 15:46:53] REQUEST <<GET /images/search?term=mjbright/docker-demo HTTP/1.1
    Host: localhost:2475
    User-Agent: curl/7.52.1
    Accept: */*>>




In [40]:
cat /tmp/dump.last.rsp

[2017-07-03 15:46:54] RESPONSE <<HTTP/1.1 200 OK
    Api-Version: 1.30
    Content-Type: application/json
    Docker-Experimental: false
    Ostype: linux
    Server: Docker/17.06.0-ce (linux)
    Date: Mon, 03 Jul 2017 13:46:54 GMT
    Content-Length: 107>>

RESPONSE=<<[
    {
        "description": "", 
        "is_automated": false, 
        "is_official": false, 
        "name": "mjbright/docker-demo", 
        "star_count": 0
    }
]>>


# Docker API - direct access

Remember that '*docker run*' actually fetches an image if not present locally.

This is the equivalent of '*docker image pull*'

In [41]:
!curl -XPOST http://localhost:2475/images/create?fromImage=mjbright/docker-demo:20  2>/dev/null| jq .

{
  "status": "Pulling from mjbright/docker-demo",
  "id": "20"
}
{
  "status": "Digest: sha256:fdfb575bdfda156d5b06c7994deb770aac0efdacc3c91f285a35da8582772487"
}
{
  "status": "Status: Image is up to date for mjbright/docker-demo:20"
}


Let's look at recent requests made to our Docker daemon

(Note the time that the request took - the Docker daemon checked with DockerHub that it has the latest image)

In [42]:
!grep -aE "GET|POST|PUT" /tmp/dump.txt | tail -12

[2017-07-03 15:45:30] REQUEST <<GET /_ping HTTP/1.1
[2017-07-03 15:45:30] REQUEST <<GET /_ping HTTP/1.1
[2017-07-03 15:45:30] REQUEST <<GET /v1.30/images/json HTTP/1.1
[2017-07-03 15:45:30] REQUEST <<GET /v1.30/images/json HTTP/1.1
[2017-07-03 15:45:55] REQUEST <<GET /_ping HTTP/1.1
[2017-07-03 15:45:55] REQUEST <<GET /_ping HTTP/1.1
[2017-07-03 15:45:55] REQUEST <<GET /v1.30/images/json?all=1 HTTP/1.1
[2017-07-03 15:45:55] REQUEST <<GET /v1.30/images/json?all=1 HTTP/1.1
[2017-07-03 15:46:53] REQUEST <<GET /images/search?term=mjbright/docker-demo HTTP/1.1
[2017-07-03 15:46:53] REQUEST <<GET /images/search?term=mjbright/docker-demo HTTP/1.1
[2017-07-03 15:48:54] REQUEST <<POST /images/create?fromImage=mjbright/docker-demo:20 HTTP/1.1
[2017-07-03 15:48:54] REQUEST <<POST /images/create?fromImage=mjbright/docker-demo:20 HTTP/1.1


In [43]:
!docker run -d -p 3333:8080 mjbright/docker-demo:20

a3e90dba01c0ce36239a621ea4fb363383c5fcbaf0a8674b5b0180cabf743207


In [44]:
cat /tmp/dump.last.req

[2017-07-03 15:50:29] REQUEST <<POST /v1.30/containers/a3e90dba01c0ce36239a621ea4fb363383c5fcbaf0a8674b5b0180cabf743207/start HTTP/1.1
    Host: 127.0.0.1:2475
    User-Agent: Docker-Client/17.06.0-ce (linux)
    Content-Length: 0
    Content-Type: text/plain
    Accept-Encoding: gzip>>




In [45]:
!curl http://localhost:3333




                                                .---------.                                          
                                               .///++++/:.                                          
                                               .///+++//:.                                          
                                               .///+++//:.                                          
                             ``````````````````.:///////:.                       `                  
                             .-///////:://+++//::///////-.                      .--.                
                             .:::///:::///+++///:::///:::.                     .:ss+-`              
                             .:::///:::///+++///:::///:::.                    `.ossss:.             
                             .:///////:/+++oo++/:///////:.                    .-ssssss:.            
                    .-:::::::--///////--:::::::--///////--:::::::-.           `-sssssso

In [46]:
!grep -a REQUEST /tmp/dump.txt | tail -12

[2017-07-03 15:46:53] REQUEST <<GET /images/search?term=mjbright/docker-demo HTTP/1.1
[2017-07-03 15:46:53] REQUEST <<GET /images/search?term=mjbright/docker-demo HTTP/1.1
[2017-07-03 15:48:54] REQUEST <<POST /images/create?fromImage=mjbright/docker-demo:20 HTTP/1.1
[2017-07-03 15:48:54] REQUEST <<POST /images/create?fromImage=mjbright/docker-demo:20 HTTP/1.1
[2017-07-03 15:50:28] REQUEST <<GET /_ping HTTP/1.1
[2017-07-03 15:50:28] REQUEST <<GET /_ping HTTP/1.1
[2017-07-03 15:50:28] REQUEST <<POST /v1.30/containers/create HTTP/1.1
[2017-07-03 15:50:28] REQUEST <<POST /v1.30/containers/create HTTP/1.1
[2017-07-03 15:50:29] REQUEST <<POST /v1.30/containers/a3e90dba01c0ce36239a621ea4fb363383c5fcbaf0a8674b5b0180cabf743207/wait?condition=next-exit HTTP/1.1
[2017-07-03 15:50:29] REQUEST <<POST /v1.30/containers/a3e90dba01c0ce36239a621ea4fb363383c5fcbaf0a8674b5b0180cabf743207/wait?condition=next-exit HTTP/1.1
[2017-07-03 15:50:29] REQUEST <<POST /v1.30/containers/a3e90dba01c0ce36239a621ea4fb3

In [47]:
!curl -XPOST http://localhost:2475/images/create?fromImage=mjbright/nosuchimage 2>/dev/null | jq .

{
  "message": "repository mjbright/nosuchimage not found: does not exist or no pull access"
}


In [48]:
!grep -a REQUEST /tmp/dump.txt | tail -4

[2017-07-03 15:50:29] REQUEST <<POST /v1.30/containers/a3e90dba01c0ce36239a621ea4fb363383c5fcbaf0a8674b5b0180cabf743207/start HTTP/1.1
[2017-07-03 15:50:29] REQUEST <<POST /v1.30/containers/a3e90dba01c0ce36239a621ea4fb363383c5fcbaf0a8674b5b0180cabf743207/start HTTP/1.1
[2017-07-03 15:53:06] REQUEST <<POST /images/create?fromImage=mjbright/nosuchimage HTTP/1.1
[2017-07-03 15:53:06] REQUEST <<POST /images/create?fromImage=mjbright/nosuchimage HTTP/1.1


In [49]:
!docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://cloud.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/engine/userguide/



In [50]:
!docker container create hello-world

b358bb76c6d1710156eeafb06e30c621494d631b6d3f2968f874ee36beb7ca18


In [51]:
!docker container start -a $(docker ps -ql)


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://cloud.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/engine/userguide/



In [52]:
!grep -aE "GET|POST|PUT" tcpdump.log | tail -12

3...3...POST /images/create?fromImage=mjbright/nosuchimage HTTP/1.1
3.-.3.-.GET /_ping HTTP/1.1
3.-.3.-.POST /v1.30/containers/create HTTP/1.1
3.1.3.1.POST /v1.30/containers/efa634d4c0dd4710d1a8b3bbc4d4ec6f88877d8c1c3c7e4a3a7f8cc16ebbc269/attach?stderr=1&stdout=1&stream=1 HTTP/1.1
3.1.3.1.POST /v1.30/containers/efa634d4c0dd4710d1a8b3bbc4d4ec6f88877d8c1c3c7e4a3a7f8cc16ebbc269/wait?condition=next-exit HTTP/1.1
3.1.3.1.POST /v1.30/containers/efa634d4c0dd4710d1a8b3bbc4d4ec6f88877d8c1c3c7e4a3a7f8cc16ebbc269/start HTTP/1.1
3.x.3.x.GET /_ping HTTP/1.1
3.x.3.x.POST /v1.30/containers/create HTTP/1.1
3..%3..%GET /_ping HTTP/1.1
3..&3..&GET /v1.30/containers/json?limit=1 HTTP/1.1
3../3../GET /_ping HTTP/1.1
3..03..0GET /v1.30/containers/b358bb76c6d1/json HTTP/1.1


In [53]:
!curl -X POST -H "User-Agent: Docker-Client/17.05.0-ce (linux)" \
              -H "Content-Type: application/json" \
              -H "Accept-Encoding: gzip" \
      http://localhost:2475/containers/create \
    -g --data '{\
        "Hostname":"","Domainname":"","User":"","AttachStdin":false,"AttachStdout":true,"AttachStderr":true,"Tty":false,"OpenStdin":false,"StdinOnce":false,"Env":[],"Cmd":null,"Image":"hello-world","Volumes":{},"WorkingDir":"","Entrypoint":null,"OnBuild":null,"Labels":{},"HostConfig":{"Binds":null,"ContainerIDFile":"","LogConfig":{"Type":"","Config":{}},"NetworkMode":"default","PortBindings":{},"RestartPolicy":{"Name":"no","MaximumRetryCount":0},"AutoRemove":false,"VolumeDriver":"","VolumesFrom":null,"CapAdd":null,"CapDrop":null,"Dns":[],"DnsOptions":[],"DnsSearch":[],"ExtraHosts":null,"GroupAdd":null,"IpcMode":"","Cgroup":"","Links":null,"OomScoreAdj":0,"PidMode":"","Privileged":false,"PublishAllPorts":false,"ReadonlyRootfs":false,"SecurityOpt":null,"UTSMode":"","UsernsMode":"","ShmSize":0,"ConsoleSize":[0,0],"Isolation":"","CpuShares":0,"Memory":0,"NanoCpus":0,"CgroupParent":"","BlkioWeight":0,"BlkioWeightDevice":null,"BlkioDeviceReadBps":null,"BlkioDeviceWriteBps":null,"BlkioDeviceReadIOps":null,"BlkioDeviceWriteIOps":null,"CpuPeriod":0,"CpuQuota":0,"CpuRealtimePeriod":0,"CpuRealtimeRuntime":0,"CpusetCpus":"","CpusetMems":"","Devices":[],"DeviceCgroupRules":null,"DiskQuota":0,"KernelMemory":0,"MemoryReservation":0,"MemorySwap":0,"MemorySwappiness":-1,"OomKillDisable":false,"PidsLimit":0,"Ulimits":null,"CpuCount":0,"CpuPercent":0,"IOMaximumIOps":0,"IOMaximumBandwidth":0},"NetworkingConfig":{"EndpointsConfig":{}}}' 

{"Id":"f036cea7c90370329d6ef6116cc09a51b334f6c82489e17970c93ee51053b111","Warnings":null}


In [54]:
!docker start -a $(docker ps -ql)


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://cloud.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/engine/userguide/



In [ ]:
!grep -aE "GET|POST|PUT" tcpdump.log | tail -12

**Below**, we can see that our container is still present, but not running

(See 2nd line:
   "Status":"exited","Running":false
)

In [ ]:
!curl -X GET  -H "User-Agent: Docker-Client/17.05.0-ce (linux)" \
              -H "Content-Type: application/json" \
              -H "Accept-Encoding: gzip" \
      http://localhost:2475/v1.29/containers/$(docker ps -ql)/json

In [ ]:
!docker version

In [ ]:
!docker start -a $(docker ps -ql)

In [ ]:
!grep -aE "GET|POST|PUT" tcpdump.log | tail -12

In [ ]:
!set -x; LAST_CONTAINER_ID=$(docker ps -ql --no-trunc); curl -X POST -H "User-Agent: Docker-Client/17.05.0-ce (linux)" \
              -H "Content-Type: application/json" \
              -H "Accept-Encoding: gzip" \
      "http://localhost:2475/v1.29/containers/$LAST_CONTAINER_ID/attach?stderr=1&stdout=1&stream=1"

# Docker-Python "Swarm Mode" Lab


<a name="MACHINE" />
# Docker-machine from Python
<a href="#TOP"> TOP </a>


Define here if we should use docker-machines as is, or delete existing machines/create new machines



In [55]:
!docker-machine ls

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.100:2376           v17.03.2-ce   
swnode1     -        virtualbox   Running   tcp://192.168.99.101:2376           v17.03.2-ce   
swnode2     -        virtualbox   Running   tcp://192.168.99.102:2376           v17.03.2-ce   


In [62]:
# CREATE_MACHINES=True
# DELETE_MACHINES=True

CREATE_MACHINES=False
DELETE_MACHINES=False

# CREATE_MACHINES=True
# DELETE_MACHINES=True

### Undo existing cluster:

In [ ]:
%%bash

press() {
    echo -e $*
}

cmd() {
    echo -e $*
    eval $*
}

leave_swarm() {
    echo "==== ${FUNCNAME[0]} $*"
    MASTER=$1; shift
    NODE=$1;   shift

    docker $(docker-machine config $MASTER) node ls | grep $NODE && {
        press "\n---- $NODE swarm leave/node rm:"
        cmd docker $(docker-machine config $NODE) swarm leave
        while docker $(docker-machine config $MASTER) node ls  | grep $NODE | grep down; do echo Waiting for $NODE to go down; sleep 1; done
        cmd docker $(docker-machine config $MASTER) node rm $NODE

        echo
        cmd docker-machine ls
        cmd docker $(docker-machine config $MASTER) node ls
    }
}

#undo_swarm swmaster1 swnode1 swnode2
undo_swarm() {
    echo "==== ${FUNCNAME[0]} $*"
    MASTER=$1; shift
    NODES="$*"

    for NODE in $NODES; do
        leave_swarm $MASTER $NODE
    done

    cmd docker $(docker-machine config $MASTER) swarm leave --force
    cmd docker $(docker-machine config $MASTER) node ls
}

undo_swarm swmaster1 swnode1 swnode2

docker $(docker-machine config swmaster1) node ls


In [56]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

# Requirements
- Docker >= 1.12 (docker-17.06-ce recommended)
- Docker machine
- Virtualbox

# Setup

## Install docker and docker-machine modules

## NOTE:
#### python-docker-machine installed from https://github.com/mjbright/python-docker-machine

Simple modification to depend upon 'docker' not 'docker-py'.

Installed using

'''python setup.py install'''


## Check that docker is installed, but not docker-py !!

In [57]:
!which pip
!pip list --format=columns | grep -i docker

!pip list --format=columns | grep docker-py || echo -e "\n\n**** ERROR: you have Docker-py installed (not Docker)!!"

/home/mjb/usr/anaconda3/bin/pip
docker                             2.4.2      
docker-pycreds                     0.2.1      
python-docker-machine              0.2.2      
docker-pycreds                     0.2.1      


In [58]:
import docker

EXPECT_DOCKER_PY_VERSION='2.4.2'

print("docker module version=" + docker.version)

if docker.version != EXPECT_DOCKER_PY_VERSION:
    print("\n\nERROR: Bad docker.py version: {} != expected {}".format(docker.version, EXPECT_DOCKER_PY_VERSION))


docker module version=2.4.2


# Using python-docker-machine wrapper module

Here we will describe use of this Python module which provides Python access to the 'docker-machine' executable on your machine

### Referencing the docker-machine executable

In [59]:
import machine
import docker

m = machine.Machine(path="/usr/local/bin/docker-machine")

In [60]:
print(m)

### Cleanup:  Delete any existing machines

In [63]:
# If you want to delete existing machines:
# !docker-machine rm -f swmaster swnode1 swnode2

if DELETE_MACHINES:
    for node in m.ls():
        if node['Name'] != '':
            print("Deleting node " + node['Name'])
            m.rm(machine=node['Name'],force=True)
            
m.ls()

[{'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swmaster1',
  'ResponseTime': '799ms',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.100:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swnode1',
  'ResponseTime': '811ms',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.101:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swnode2',
  'ResponseTime': '799ms',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.102:2376'},
 {'Name': ''}]

### Creating a new machine

If creating a machine in some cloud provider you will need to first export some appropriate token or subscription id as
an appropriately named environment variable, e.g.
- DIGITALOCEAN_ACCESS_TOKEN for DigitalOcean
- AZURE_SUBSCRIPTION_ID for Microsoft Azure

Refer to https://docs.docker.com/machine/get-started-cloud/ for information,

and to https://docs.docker.com/machine/drivers/gce/ (select cloud provider in left-hand sidebar) for specifics.

**WARNING**: You may need to "logon" to the platform before using the python-docker-machine otherwise the python-docker-machine module will block asking you to validate - but **you will not see that message** !
** This will not work with Azure ** because of the need to logon from the local terminal

## Examples:

### Digital Ocean:  
Set DIGITALOCEAN_ACCESS_TOKEN as an environment variable 

e.g.
```
     os.environ['DIGITALOCEAN_ACCESS_TOKEN']='MY-TOKEN'
     m.create('swmaster1', driver='digitalocean', blocking=True)
```

### Azure:
Set AZURE_SUBSCRIPTION_ID as an environment variable 

e.g.
```
    os.environ['AZURE_SUBSCRIPTION_ID']='MY-ID'
    m.create('swmaster1', driver='azure', blocking=True)
```

### Google Compute Engine:
Use "gcloud auth login"

e.g.
```
     m.create('swmaster1', driver='google', blocking=True)
```

## Example of local machine creation

Below is an example of machine creation on a local machine, to be adapted as above to run in the cloud

#### NOTE: Temporary errors from "*docker-machine ls*"

From the command-line you can see the progress of machine creation, using
    ```$ docker-machine ls```
    
You may see errors during creation of machines before ssh connectivity is established and before docker host is started.

![](images/docker-machine-errors.png)

In [64]:
m.create('test-machine', driver='virtualbox', blocking=True)

m.ls()

[{'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swmaster1',
  'ResponseTime': '1.125s',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.100:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swnode1',
  'ResponseTime': '1.125s',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.101:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swnode2',
  'ResponseTime': '1.142s',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.102:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'test-machine',
  'Respon

## Deletion of a local machine

In [ ]:
!docker-machine ls

## Below is how we would delete our test-machine, but we will keep it for the moment ...

In [ ]:
# m.rm(machine='test-machine')

m.ls()

In [ ]:
!docker-machine ls

# Creating a Docker client connection to the Docker engine

During this lab we will use the installed Docker Client to talk to your Docker hosts.

Those hosts may be running 
- locally      controlled via docker-machine
- in the cloud controlled via docker-machine
- under Docker Toolbox controlled via docker-machine (https://docs.docker.com/machine/get-started/#create-a-machine)
- under Docker For Windows, controlled via docker-machine (https://docs.docker.com/docker-for-windows/#docker-settings)
- Docker for Mac, where you will only be able to have a single-node swarm (unless bursting out to cloud)

This notebook demonstrates the use of docker-machine to control the hosts.

#### NOTE: The following command is specific to my environment as I have a special DOCKER_HOST value set:

## Connection to a local Docker host

#### NOTE: This is for information, in our lab we will obtain a client connection via the python-docker-machine module, see below.

In [65]:
# NOTE: specific to my environment
#       if you have some special DOCKER_HOST setting, then check this and set in the Python code below

! echo $DOCKER_HOST

tcp://127.0.0.1:2475


### Inspecting the DockerClient interface

Let's look at what objects/methods are available using IPython introspection

We can see that we have high-level objects in the same way the "docker cli client"

## Docker-py has 2 interfaces

- low-level via Docker.APIClient interface
  - This lower-level API corresponds to the actual API requests
  - Use it only when you need it's extra features
- high-level via Docker.DockerClient interface
  - This higher-level API is what you should generally use


### Inspecting the 'low-level' APIClient interface

Let's look at what objects/methods are available using IPython introspection

In [66]:
testlo = docker.APIClient(**m.config(machine='test-machine'))

In [ ]:
testlo.

In [ ]:
testlo??

## So now let's discard our test-machine

In [67]:
m.rm(machine='test-machine')

m.ls()

[{'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swmaster1',
  'ResponseTime': '798ms',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.100:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swnode1',
  'ResponseTime': '811ms',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.101:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swnode2',
  'ResponseTime': '798ms',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.102:2376'},
 {'Name': ''}]

## We will use the 'high-level' DockerClient interface

Just as the docker cli has commands (since Docker 1.13) of the form
    - docker images list
    - docker containers list

The DockerClient interface has a similar object-oriented interface,

e.g. to list images

In [ ]:
docker.client.

#### NOTE: You may comment the 'DOCKER_HOST' lines below if you have no special configuration:

In [68]:
import docker, os

# Open a client connection to your local Docker host, set DOCKER_HOST if you have special settings to use:

# Use your DOCKER_HOST value here:
# DOCKER_HOST='tcp://127.0.0.1:2475'

#   os.environ['DOCKER_HOST']=DOCKER_HOST
# e.g.
#   os.environ['DOCKER_HOST']="tcp://127.0.0.1:2475"

# auto: use same api version as docker daemon
client = docker.from_env(version='auto') #, url='tcp://127.0.0.1:2475')

Let's look at the api of our client and compare it to the Docker cli client

In [ ]:
client.

In [69]:
!docker


Usage:	docker COMMAND

A self-sufficient runtime for containers

Options:
      --config string      Location of client config files (default
                           "/home/mjb/.docker")
  -D, --debug              Enable debug mode
      --help               Print usage
  -H, --host list          Daemon socket(s) to connect to
  -l, --log-level string   Set the logging level
                           ("debug"|"info"|"warn"|"error"|"fatal")
                           (default "info")
      --tls                Use TLS; implied by --tlsverify
      --tlscacert string   Trust certs signed only by this CA (default
                           "/home/mjb/.docker/ca.pem")
      --tlscert string     Path to TLS certificate file (default
                           "/home/mjb/.docker/cert.pem")
      --tlskey string      Path to TLS key file (default
                           "/home/mjb/.docker/key.pem")
      --tlsverify          Use TLS and verify the remote
  -v, --version            Pri

In [ ]:
!docker

In [70]:
!docker images

REPOSITORY                         TAG                 IMAGE ID            CREATED             SIZE
mjbright/cherrpyapp                latest              eb2c01b4f1fb        22 hours ago        429MB
mjbright/flaskapp                  latest              2d4ffd4a9bbc        22 hours ago        430MB
<none>                             <none>              1618df99522d        22 hours ago        424MB
<none>                             <none>              f9d927d6e26b        22 hours ago        424MB
<none>                             <none>              608fb355fab9        24 hours ago        429MB
<none>                             <none>              9909d5c82237        25 hours ago        429MB
<none>                             <none>              6d1763b21b35        25 hours ago        597MB
<none>                             <none>              cc347f6d4c45        4 days ago          429MB
<none>                             <none>              16d368958fc4        4 days ago       

In [71]:
client.images.list??

In [72]:
for image in client.images.list():
    print(image.tags)
    # was image.tag() in docker-py 2.2

['mjbright/cherrpyapp:latest']
['mjbright/flaskapp:latest']
[]
[]
[]
[]
[]
[]
[]
['projectunik/binary:latest']
['unikernel/mirage:latest']
['functions/webhookstash:latest']
['ubuntu:16.04', 'ubuntu:latest']
['hello-world:latest']
['quay.io/prometheus/alertmanager:latest']
['alexellis2/faas-nodeinfo:latest']
['centos:7']
['functions/gateway:0.5.5']
['astefanutti/decktape:latest']
['functions/alpine:health']
['mjbright/docker-demo:20']
['mjbright/docker-demo:21']
['alexellis2/faas-markdownrender:latest']
['alexellis2/faas-dockerhubstats:latest']
['projectunik/image-creator:fa560ac11cecae33']
['projectunik/boot-creator:d0665c8793b16c51']
['projectunik/compilers-rump-go-hw:d1bfbc13602e306d']
['projectunik/qemu-util:6f5922561bbb86e3']
[]
['includeos/build:latest']


In [73]:
cat /tmp/dump.last.req

[2017-07-03 16:13:47] REQUEST <<GET /v1.30/images/json?only_ids=0&all=0 HTTP/1.1
    Host: 127.0.0.1:2475
    User-Agent: docker-sdk-python/2.4.2
    Accept-Encoding: gzip, deflate
    Accept: */*
    Connection: keep-alive>>




In [74]:
cat /tmp/dump.last.rsp

In [75]:
!tail -40 /tmp/dump.txt

  {
  "Containers":-1,
    "Created":1499012058,
    "Id":"sha256:2d4ffd4a9bbc404da4f1d5a1383d639b85757b85b3fdea22e4e769a998d8da07",
    "Labels":{
    "MAINTAINER":"Mike.Bright.dockerfile@mjbright.net"}
      ,
    "ParentId":"sha256:12cdb6f03a53ef9dad2c370574a0fc93d4d7818863dd1070f9fdad79b751e37b",
    "RepoDigests":null,
    "RepoTags":[
    "mjbright/flaskapp:latest"]
      ,
    "SharedSize":-1,
    "Size":430002280,
    "VirtualSize":430002280}
    ,
  {
  "Containers":-1,
    "Created":1499011914,
    "Id":"sha256:1618df99522d02589f79757aeaa3e648169ecf096c2681b025638a5b38019096",
    "Labels":{
    "MAINTAINER":"Mike.Bright.dockerfile@mjbright.net"}
      ,
    "ParentId":"sha256:51c4db36528cd50accc0856031cdec4e26d660be57ef0e63b44d8de8f4accf97",
    "RepoDigests":[
    "<none>@<none>"]
      ,
    "RepoTags":[
    "<none>:<none>"]
      ,
    "SharedSize":-1,
    "Size":424281145,
    "VirtualSize":424281145}
    ,
  {
  "Containers":-1,
    "Created":1499011859,
    "Id":"sha25

In [ ]:
client.containers.list()

In [77]:
!docker ps 

CONTAINER ID        IMAGE                     COMMAND                  CREATED             STATUS              PORTS                    NAMES
a3e90dba01c0        mjbright/docker-demo:20   "/bin/docker-demo ..."   24 minutes ago      Up 24 minutes       0.0.0.0:3333->8080/tcp   priceless_wescoff
b0991896976d        mjbright/cherrpyapp       "python cherrypy_a..."   About an hour ago   Up About an hour    0.0.0.0:9191->8080/tcp   quizzical_yonath
ecf09b28660f        mjbright/flaskapp         "python flask_app.py"    About an hour ago   Up About an hour    0.0.0.0:5000->5000/tcp   cocky_cori


In [78]:
# Note tcpdump/scapy output whilst we pull the image ...
#      we see docker engine informing client of progress of pull ...

client.images.pull("mjbright/docker-demo:21")

<Image: 'mjbright/docker-demo:21'>

In [79]:
client.containers.run("ubuntu:latest", command="bash -c 'echo hello $(date)'")

b'hello Mon Jul 3 14:15:49 UTC 2017\n'

In [80]:
client.containers.run("ubuntu:latest", command="bash -c 'while true; do echo hello  $(date); sleep 1; done'", detach=True)

<Container: f69604ad04>

In [81]:
client.containers.list()

[<Container: f69604ad04>,
 <Container: a3e90dba01>,
 <Container: b099189697>,
 <Container: ecf09b2866>]

In [82]:
container = client.containers.list()[0]
# container = client.containers.get(  "0xid" )

In [83]:
container.id

'f69604ad041d190f0fd64cfd534f4e77e6d855d0f80fd405ac9e75573f1bdd47'

In [85]:
container.logs()

b'hello Mon Jul 3 14:16:12 UTC 2017\nhello Mon Jul 3 14:16:13 UTC 2017\nhello Mon Jul 3 14:16:14 UTC 2017\nhello Mon Jul 3 14:16:15 UTC 2017\nhello Mon Jul 3 14:16:16 UTC 2017\nhello Mon Jul 3 14:16:17 UTC 2017\nhello Mon Jul 3 14:16:18 UTC 2017\nhello Mon Jul 3 14:16:19 UTC 2017\nhello Mon Jul 3 14:16:20 UTC 2017\nhello Mon Jul 3 14:16:21 UTC 2017\nhello Mon Jul 3 14:16:23 UTC 2017\nhello Mon Jul 3 14:16:24 UTC 2017\nhello Mon Jul 3 14:16:25 UTC 2017\nhello Mon Jul 3 14:16:26 UTC 2017\nhello Mon Jul 3 14:16:27 UTC 2017\nhello Mon Jul 3 14:16:28 UTC 2017\nhello Mon Jul 3 14:16:29 UTC 2017\nhello Mon Jul 3 14:16:30 UTC 2017\nhello Mon Jul 3 14:16:31 UTC 2017\nhello Mon Jul 3 14:16:32 UTC 2017\nhello Mon Jul 3 14:16:33 UTC 2017\nhello Mon Jul 3 14:16:34 UTC 2017\nhello Mon Jul 3 14:16:35 UTC 2017\nhello Mon Jul 3 14:16:36 UTC 2017\nhello Mon Jul 3 14:16:37 UTC 2017\nhello Mon Jul 3 14:16:38 UTC 2017\nhello Mon Jul 3 14:16:39 UTC 2017\nhello Mon Jul 3 14:16:40 UTC 2017\nhello Mon Jul 3 14

In [86]:
container.stop()

In [87]:
container.remove()

In [88]:
client.containers.list()

[<Container: a3e90dba01>, <Container: b099189697>, <Container: ecf09b2866>]

In [89]:
client.containers.list(all=True,limit=200)

[<Container: 5bcc2a339a>,
 <Container: f036cea7c9>,
 <Container: b358bb76c6>,
 <Container: efa634d4c0>,
 <Container: a3e90dba01>,
 <Container: 58c1a1d09e>,
 <Container: b099189697>,
 <Container: ecf09b2866>]

In [90]:
for container in client.containers.list(all=True):
    container.stop()
    container.remove()

In [91]:
client.containers.list(all=True,limit=200)

[]

In [ ]:
cat /tmp/dump.last.req

In [ ]:
cat /tmp/dump.last.rsp

# Create nodes for your swarm cluster (1 master and 2 worker nodes)

We will create 3 nodes using docker-machine/virtualbox.

![](images/DockerSwarmArchi.svg)

In [92]:
#### If not already created, we can create our machines here:
import datetime
import time
import os

def createNodes(nodes):
    for node in nodes:
        NOW=datetime.datetime.now().strftime("%H:%M:%S")
        SECS_BEFORE=int(time.time())
        print("{} Creating node {} <on {}> ...".format(NOW, node, MACHINE_DRIVER))
        
        m.create(node, driver=MACHINE_DRIVER, blocking=False)
        
        SECS_AFTER=int(time.time())
        print("... took {} secs".format(SECS_AFTER-SECS_BEFORE))

MACHINE_DRIVER='virtualbox'
MASTER_NODES=['swmaster1']
WORKER_NODES=['swnode1','swnode2']

In [93]:
if CREATE_MACHINES:
    createNodes(MASTER_NODES + WORKER_NODES)

## Recovering from possible docker-machine/VirtualBox problems.

It's possible that this machine creation fails, e.g. with error as below:

```
RuntimeError: cmd returned error 1: Error creating machine: Error in driver during machine creation: /usr/bin/VBoxManage modifyvm swmaster1 --natpf1 ssh,tcp,127.0.0.1,40886,,22 failed:
VBoxManage: error: The machine 'swmaster1' is already locked for a session (or being unlocked)
VBoxManage: error: Details: code VBOX_E_INVALID_OBJECT_STATE (0x80bb0007), component MachineWrap, interface IMachine, callee nsISupports
VBoxManage: error: Context: "LockMachine(a->session, LockType_Write)" at line 493 of file VBoxManageModifyVM.cpp
```

To recover, list current VMs known by VirtualBox using
```VBoxManage list vms```

For any VMs which are marked as 'inaccessible', delete them using their ID, e.g.
```VBoxManage unregistervm 576f28c7-796e-47d4-a1b1-55dada4bba14```

If other machines are listed in error seen by
```docker-machine ls```

try to delete theme with
```docker-machine rm -f <machine-name>```

If this fails, perform a "VBoxManage unregistervm", of the VM id or it's name
e.g.
```VBoxManage unregistervm 576f28c7-796e-47d4-a1b1-55dada4bba14```
or
```VBoxManage unregistervm swmaster1```

Then try to create machines again

In [94]:
m.ls()

[{'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swmaster1',
  'ResponseTime': '777ms',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.100:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swnode1',
  'ResponseTime': '789ms',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.101:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.03.2-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swnode2',
  'ResponseTime': '777ms',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.102:2376'},
 {'Name': ''}]

We can obtain the "config" parameters (o/p of "docker-machine config <machine name>")

In [95]:
m.config(machine='swmaster1')

{'base_url': 'https://192.168.99.100:2376',
 'tls': <docker.tls.TLSConfig at 0x7fc774e56d30>}

We can obtain the "env" parameters (o/p of "docker-machine env <machine name>")

In [96]:
m.env(machine='swmaster1')

['export',
 'DOCKER_TLS_VERIFY="1"',
 'export',
 'DOCKER_HOST="tcp://192.168.99.100:2376"',
 'export',
 'DOCKER_CERT_PATH="/home/mjb/.docker/machine/machines/swmaster1"',
 'export',
 'DOCKER_MACHINE_NAME="swmaster1"',
 '#',
 'Run',
 'this',
 'command',
 'to',
 'configure',
 'your',
 'shell:',
 '#',
 'eval',
 '$(/usr/local/bin/docker-machine',
 'env',
 'swmaster1)']

<a name="DOCKERPY" />
# docker-py: Controlling Docker from Python
<a href="#TOP"> TOP </a>



## Creating DockerClients (docker-py)

Now let's create docker-py client objects for each of our new nodes

In [97]:
# Create Docker clients for each machine:
nodes={}

swmaster1 = docker.DockerClient(**m.config(machine='swmaster1'))

nodes['swmaster1']=swmaster1
swnode1 = docker.DockerClient(**m.config(machine='swnode1'))
nodes['swnode1']=swnode1
swnode2 = docker.DockerClient(**m.config(machine='swnode2'))
nodes['swnode2']=swnode2

pp.pprint(nodes)

{   'swmaster1': <docker.client.DockerClient object at 0x7fc774e5a208>,
    'swnode1': <docker.client.DockerClient object at 0x7fc774ef5c50>,
    'swnode2': <docker.client.DockerClient object at 0x7fc774ed9550>}


In [98]:
# Check connectivity to machines:

print("Checking that nodes are 'ping'able")

for node in nodes.keys():
    print("{}: {}".format(node, nodes[node].ping()))

Checking that nodes are 'ping'able
swmaster1: True
swnode1: True
swnode2: True


In [99]:
swmaster1.version()

{'ApiVersion': '1.27',
 'Arch': 'amd64',
 'BuildTime': '2017-06-27T01:35:00.165278597+00:00',
 'GitCommit': 'f5ec1e2',
 'GoVersion': 'go1.7.5',
 'KernelVersion': '4.4.74-boot2docker',
 'MinAPIVersion': '1.12',
 'Os': 'linux',
 'Version': '17.03.2-ce'}

In [100]:
swmaster1.images.list()

[]

In [ ]:
swmaster1??

# swarm init

Now that we have 3 nodes available, we will initialize our Swarm Cluster with 1 master node.

We've initially created 3 nodes, but we do not yet have a Swarm cluster created.

![](images/SwarmNodes_3nodes.png)

Including these parameters on the docker command line will connect the client to the docker daemon running on node '*swmaster*'.

### Networks before creation of swarm cluster
Before going further let's look at the networks on your machine.

Later, we'll see how a new network is created once the swarm cluster has been created.

In [101]:
def show_networks(client):
    FORMAT_STR="%-14s %-20s %-10s %-14s"
    print(FORMAT_STR % ("NETWORK ID", "NAME", "DRIVER", "SCOPE"))

    for network in client.networks.list():
        #print(dir(network))
        print(FORMAT_STR % (network.short_id, network.name, network.attrs['Driver'], network.attrs['Scope']))

In [102]:
  
# NETWORK ID          NAME                DRIVER              SCOPE
# ab973c7b9062        bridge              bridge              local
# e953fc8fe3f2        host                host                local
# d45eb5315d7c        none                null                local

show_networks(swmaster1)

NETWORK ID     NAME                 DRIVER     SCOPE         
75062ff3ec     host                 host       local         
49db0f8876     bridge               bridge     local         
dd8ce398ea     none                 null       local         


### Obtain ip addresses of nodes, esp. the 1st master node

Now let's identify the ip address of our nodes.

We need to note the ip address of the master node, so that when we initialize our swarm cluster, this node will advertise itself on this address so that other nodes can join the swarm cluster.

We can see this through config or ip commands of docker-machine as shown below.

In [103]:
for node in nodes.keys():
    ip = m.ip(machine=node)
    print("{}: {}".format(node, ip))

master1ip = m.ip(machine='swmaster1')

swmaster1: 192.168.99.100
swnode1: 192.168.99.101
swnode2: 192.168.99.102


We could then provide the above ip address as parameter to --advertise-addr when initializing the swarm.

However, it is quite convenient to run the above commands embedded, as below, as arguments to the swarm init command.

docker-machine config swmaster provides the parameters to use when connecting to the appropriate docker engine for our machine "swmaster".

The following command will run swarm init to generate the cluster with 'swmaster' as the Master node.
You should see output similar to the below:

In [104]:
swmaster1.info()

{'Architecture': 'x86_64',
 'BridgeNfIp6tables': True,
 'BridgeNfIptables': True,
 'CPUSet': True,
 'CPUShares': True,
 'CgroupDriver': 'cgroupfs',
 'ClusterAdvertise': '',
 'ClusterStore': '',
 'ContainerdCommit': {'Expected': '4ab9917febca54791c5f071a9d1f404867857fcc',
  'ID': '4ab9917febca54791c5f071a9d1f404867857fcc'},
 'Containers': 0,
 'ContainersPaused': 0,
 'ContainersRunning': 0,
 'ContainersStopped': 0,
 'CpuCfsPeriod': True,
 'CpuCfsQuota': True,
 'Debug': True,
 'DefaultRuntime': 'runc',
 'DockerRootDir': '/mnt/sda1/var/lib/docker',
 'Driver': 'aufs',
 'DriverStatus': [['Root Dir', '/mnt/sda1/var/lib/docker/aufs'],
  ['Backing Filesystem', 'extfs'],
  ['Dirs', '0'],
  ['Dirperm1 Supported', 'true']],
 'ExperimentalBuild': False,
 'HttpProxy': '',
 'HttpsProxy': '',
 'ID': '2DJW:E6MO:VV3H:RZNM:KE6E:JR6F:F3DM:VZ4U:ZMJX:HIY7:YYLA:Y32J',
 'IPv4Forwarding': True,
 'Images': 0,
 'IndexServerAddress': 'https://index.docker.io/v1/',
 'InitBinary': 'docker-init',
 'InitCommit': {'Ex

In [105]:
swmaster1.info()['Swarm']

{'Cluster': {'CreatedAt': '0001-01-01T00:00:00Z',
  'ID': '',
  'Spec': {'CAConfig': {},
   'Dispatcher': {},
   'EncryptionConfig': {'AutoLockManagers': False},
   'Orchestration': {},
   'Raft': {'ElectionTick': 0, 'HeartbeatTick': 0},
   'TaskDefaults': {}},
  'UpdatedAt': '0001-01-01T00:00:00Z',
  'Version': {}},
 'ControlAvailable': False,
 'Error': '',
 'LocalNodeState': 'inactive',
 'Managers': 0,
 'NodeAddr': '',
 'NodeID': '',
 'Nodes': 0,
 'RemoteManagers': None}

### Before creating the swarm cluster we have 0 nodes, and an inactive cluster

![](images/SwarmNodes_3nodes.png)


In [106]:
if 'Nodes' in swmaster1.info()['Swarm']:
    swmaster1.info()['Swarm']['Nodes']

In [107]:
swmaster1.info()['Swarm']['LocalNodeState']

'inactive'

In [ ]:
# Note: the following is specific to the Jupyter notebook environment allowing to introspect class/method/variable definitions:

swmaster1.swarm.init?

Now let's initialize our swarm mode.

This make this node a 'Manager' node of the cluster and creates new overlay network to be used by services running in the cluster.

In [108]:
swmaster1.swarm.init(advertise_addr=master1ip)


We now have a Swarm cluster with 1 Master node.

![](images/SwarmNodes_3nodes_1m.png)

Let's investigate

In [109]:
swarm_info = swmaster1.swarm.client.info()

#pp.pprint(swarm_info)

pp.pprint(swarm_info['Swarm'])

{   'Cluster': {   'CreatedAt': '2017-07-03T14:26:24.577425478Z',
                   'ID': '4u7032rg0zsazvjzfcfoccsex',
                   'Spec': {   'CAConfig': {'NodeCertExpiry': 7776000000000000},
                               'Dispatcher': {'HeartbeatPeriod': 5000000000},
                               'EncryptionConfig': {'AutoLockManagers': False},
                               'Name': 'default',
                               'Orchestration': {   'TaskHistoryRetentionLimit': 5},
                               'Raft': {   'ElectionTick': 3,
                                           'HeartbeatTick': 1,
                                           'KeepOldSnapshots': 0,
                                           'LogEntriesForSlowFollowers': 500,
                                           'SnapshotInterval': 10000},
                               'TaskDefaults': {}},
                   'UpdatedAt': '2017-07-03T14:26:25.201614598Z',
                   'Version': {'Index': 10}},
  

### After creating the swarm cluster we have 1 nodes, and an active cluster


![](images/SwarmNodes_3nodes_1m.png)



In [110]:
swmaster1.info()['Swarm']['Nodes']

1

In [111]:
swmaster1.info()['Swarm']['RemoteManagers']

[{'Addr': '192.168.99.100:2377', 'NodeID': 'x3f5shhedzq0ff0hqcq5xxole'}]

A docker info should now show "Swarm: active" as below:

In [112]:
swmaster1.info()['Swarm']['LocalNodeState']

'active'

### Networks created in swarm mode

If we look at the networks we should now see new networks such as '*ingress*' an overlay network and docker_gwbridge for the swarm cluster.

In [113]:
show_networks(swmaster1)
#show_networks(swmaster2)

NETWORK ID     NAME                 DRIVER     SCOPE         
jsux5dng3b     ingress              overlay    swarm         
7ebbf5e663     docker_gwbridge      bridge     local         
75062ff3ec     host                 host       local         
49db0f8876     bridge               bridge     local         
dd8ce398ea     none                 null       local         


### We can obtain the join tokens allowing to join new nodes as manager or worker nodes

In [114]:
join_tokens = swmaster1.swarm.attrs['JoinTokens']

pp.pprint(join_tokens)

manager_token = join_tokens['Manager']
worker_token = join_tokens['Worker']

{   'Manager': 'SWMTKN-1-5wapsokhrncxkao84y7jj5pir3iz6vgf2fpfybwk2kq3xelovi-9i9h7zzmw3vm7zpgbq59avx3q',
    'Worker': 'SWMTKN-1-5wapsokhrncxkao84y7jj5pir3iz6vgf2fpfybwk2kq3xelovi-a3qxt8jedveu1y1vjkx89feki'}


# swarm join

Now we wish to join Master and Worker nodes to our swarm cluster, to do this we need to obtain the token generated during the "swarm init".

In [ ]:
print(manager_token)

In [ ]:
pp.pprint(swmaster1.info()['Swarm'])

In [ ]:
print(master1ip)

## Add 2nd, 2rd master nodes if present

In [ ]:
#if swmaster2 != None:
#    swmaster2.swarm.join(join_token=manager_token, remote_addrs=[master1ip+':2377'],
#                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

In [ ]:
#if swmaster3 != None:
#    swmaster3.swarm.join(join_token=manager_token, remote_addrs=[master1ip+':2377'],
#                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

In [ ]:
!docker $(docker-machine config swmaster1) node list

In [115]:
def show_nodes(client):
    FORMAT_STR="%-14s %-10s %-10s %-14s %-10s"
    print(FORMAT_STR % ("ID", "HOSTNAME", "STATE", "AVAILABILITY", "MANAGER STATUS"))
    
    for node in client.nodes.list():
        #pp.pprint(node.attrs)
        nodeSwarmStatus=''
        state = node.attrs['Status']['State']
        if 'ManagerStatus' in node.attrs:
            if 'Leader' in node.attrs['ManagerStatus'] and node.attrs['ManagerStatus']['Leader']:
                nodeSwarmStatus='leader'
            elif 'Reachability' in node.attrs['ManagerStatus']:
                nodeSwarmStatus=node.attrs['ManagerStatus']['Reachability']
        print(FORMAT_STR % (node.id[:12], node.attrs['Description']['Hostname'], state, node.attrs['Spec']['Availability'], nodeSwarmStatus))

        
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
x3f5shhedzq0   swmaster1  ready      active         leader    


## Add worker nodes

In [116]:
print(worker_token)

SWMTKN-1-5wapsokhrncxkao84y7jj5pir3iz6vgf2fpfybwk2kq3xelovi-a3qxt8jedveu1y1vjkx89feki


Now we can use this token to join nodes as a worker to this cluster

Note: we could also join nodes as Master, but we have only 3 nodes available.

Let's join swnode1 as a worker node

In [ ]:
import json

json_nodes = swmaster1.nodes.list()

#print(type(json_nodes))

for node in json_nodes:
    pp.pprint("node {}.attrs=".format(node.attrs['Description']['Hostname']))
    pp.pprint(node.attrs)
    print()
    

In [117]:
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
x3f5shhedzq0   swmaster1  ready      active         leader    


In [118]:
swmaster1.nodes.list()

[<Node: x3f5shhedz>]

In [ ]:
# Note: the following is specific to the Jupyter notebook environment allowing to introspect class/method/variable definitions:

swnode1.swarm.join?

In [119]:
swnode1.swarm.join(join_token=worker_token, remote_addrs=[master1ip+':2377'],
                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

True

We now have 1 Master node and 1 Worker node

![](images/SwarmNodes_3nodes_1m_1w.png)



In [120]:
swmaster1.nodes.list()

[<Node: ihnkg8j6yk>, <Node: x3f5shhedz>]

In [ ]:
!docker-machine ls

In [121]:
!docker $(docker-machine config swmaster1) node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
ihnkg8j6yk0049nj6xhtuwjog     swnode1             Ready               Active              
x3f5shhedzq0ff0hqcq5xxole *   swmaster1           Ready               Active              Leader


In [122]:
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
ihnkg8j6yk00   swnode1    ready      active                   
x3f5shhedzq0   swmaster1  ready      active         leader    


In [123]:
swnode2.swarm.join(join_token=worker_token, remote_addrs=[master1ip+':2377'],
                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

True

We now have 1 Master node and 2 Worker nodes

![](images/SwarmNodes_3nodes_1m_2w.png)



In [124]:
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
ihnkg8j6yk00   swnode1    ready      active                   
me1v63cowcn0   swnode2    ready      active                   
x3f5shhedzq0   swmaster1  ready      active         leader    


In [125]:
!docker $(docker-machine config swmaster1) node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
ihnkg8j6yk0049nj6xhtuwjog     swnode1             Ready               Active              
me1v63cowcn0djv0lu3zh8noe     swnode2             Ready               Active              
x3f5shhedzq0ff0hqcq5xxole *   swmaster1           Ready               Active              Leader


# start service

First we check for any running services - there should be none in our newly initialized cluster:

In [126]:
swmaster1.services.list()

[]

Now we will create a new service based on the docker image mariolet/docker-demo

We will expose this service on port 8080


In [ ]:
swmaster1.services.create??

In [ ]:
docker.types.ContainerSpec?

## Create a service

Now let us use the Client API to create a new service

Note how we create ContainerSpec and TaskTemplate types wich we pass to the Client.Services.create() method

Here is a first version, which launches an alpine image which runs for 60 seconds:

In [127]:
IMAGE = 'ubuntu'

container_spec = docker.types.ContainerSpec(
    IMAGE, ['echo', 'hello']
)

"""
endpoint_spec = docker.types.EndpointSpec(ports={
    12357: (1990, 'udp'),cv
    12562: (678,),
    53243: 8080,
})
"""
endpoint_spec = docker.types.EndpointSpec(ports={
    4040: 8080,
})

task_tmpl = docker.types.TaskTemplate(container_spec)

In [133]:
IMAGE = 'alpine'

container_spec = docker.types.ContainerSpec(
    IMAGE, ['echo', 'hello']
)

"""
endpoint_spec = docker.types.EndpointSpec(ports={
    12357: (1990, 'udp'),cv
    12562: (678,),
    53243: 8080,
})
"""
endpoint_spec = docker.types.EndpointSpec(ports={
    4040: 8080,
})

task_tmpl = docker.types.TaskTemplate(container_spec)

In [ ]:
swmaster1.services.create??

In [134]:
#swmaster1.services.create(task_tmpl, name='docker-demo', endpoint_spec=endpoint_spec)

service = swmaster1.services.create(
            # create arguments
            name='alpine-test',
            labels={'foo': 'bar'},
            # ContainerSpec arguments
            image=IMAGE,
            command="sleep 20",
            container_labels={'container': 'label'}
)

In [129]:
pp.pprint( swmaster1.services.list() )

!echo "Sleeping ..."; sleep 30



[<Service: p93mgwtydb>]
Sleeping ...


after 20 secs ...

We might expect that after 1 minute, the service is no longer running.

But in fact adter 1 minute although the container running the sleep command stops the service is still running and recognizing that the container exited it will restart a new container ("*task*") for this service


In [130]:
pp.pprint( swmaster1.services.list() )


[<Service: p93mgwtydb>]


In [131]:
service = swmaster1.services.list()[0]
print("Service=" + str(service) )
service_id = service.id

print("\nService attributes=")
pp.pprint((service.attrs) )

print("\nTasks associated with the service:")
for task in service.tasks():
    print("\n---- Task:")
    pp.pprint(task)

Service=<Service: p93mgwtydb>

Service attributes=
{   'CreatedAt': '2017-07-03T14:34:10.535070457Z',
    'Endpoint': {'Spec': {}},
    'ID': 'p93mgwtydbrp260d9nprdburu',
    'Spec': {   'Labels': {'foo': 'bar'},
                'Mode': {'Replicated': {'Replicas': 1}},
                'Name': 'ubuntu-test',
                'TaskTemplate': {   'ContainerSpec': {   'Command': [   'sleep',
                                                                        '20'],
                                                         'Image': 'ubuntu',
                                                         'Labels': {   'container': 'label'}},
                                    'ForceUpdate': 0}},
    'UpdateStatus': {   'CompletedAt': '0001-01-01T00:00:00Z',
                        'StartedAt': '0001-01-01T00:00:00Z'},
    'UpdatedAt': '2017-07-03T14:34:10.535070457Z',
    'Version': {'Index': 21}}

Tasks associated with the service:

---- Task:
{   'CreatedAt': '2017-07-03T14:35:06.278461781Z',

Now let's remove this service, before creating other services

In [132]:
service.remove()

True

In [ ]:
print("Running containers on master node:" + str(swmaster1.containers.list()) )

print("Running services on master node:" + str(swmaster1.services.list()))

#swmaster1.services.create(image=IMAGE, name='docker-demo', labels={'label1': 'label2'}, command=None)

# Launching "*docker-demo*" as a service

In [ ]:
IMAGE = 'mjbright/docker-demo:20'

container_spec = docker.types.ContainerSpec(
#    IMAGE, ['echo', 'hello']
    IMAGE, ['/bin/docker-demo', '-listen=:8080']
)

endpoint_spec = docker.types.EndpointSpec(ports={
    3030: 8080,
})

#task_tmpl = docker.types.TaskTemplate(container_spec)

docker_demo_service = swmaster1.services.create(
            # create arguments
            name='docker-demo',
            labels={'label1': 'value1'},
            # ContainerSpec arguments
            image=IMAGE,
            #command="sleep 60",
            container_labels={'container': 'label'},
            endpoint_spec=endpoint_spec
        )


In [ ]:
#BUT PORTS NOT SHOWING:
    
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service ps docker-demo

In [ ]:
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service create --replicas 1 --name docker-demo-via-cli -p 9090:8080 mariolet/docker-demo:20

In [ ]:
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service ps docker-demo-via-cli
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service rm docker-demo-via-cli

Now we list services again and we should see our newly added docker-demo service

In [ ]:
swmaster1.services.list()

In [ ]:
def showServices(master):
    for service in master.services.list():
        print("{}: {}".format(service, service.name))

showServices(swmaster1)

service = swmaster1.services.list()[0]
service_id = service.id

In [ ]:
#swmaster1.services.get(service_id=service_id).remove()
#showServices(swmaster1)

In [ ]:
#!docker $(docker-machine config swmaster1) service rm docker-demo

In [ ]:
!docker $(docker-machine config swmaster1) service ls

... and we can look at the service as seen by the cluster:

In [ ]:
!docker $(docker-machine config swmaster1) service ps docker-demo

... and we can look at the service on the individual cluster nodes.

Of course as we set replicas to 1 there is only 1 replica of the service for the moment, running on just 1 node of our cluster:

In [ ]:
service = swmaster1.services.list()[0]

service.tasks()

In [ ]:
print("tasks(containers) running on node swmaster1:" + str(swmaster1.containers.list()) )
print("tasks(containers) running on node swnode1:" + str(swnode1.containers.list()) )
print("tasks(containers) running on node swnode2:" + str(swnode2.containers.list()) )

In [ ]:
!docker $(docker-machine config swmaster1) ps

In [ ]:
!docker $(docker-machine config swnode1) ps

In [ ]:
!docker $(docker-machine config swnode2) ps

# Accessing the service

As we are working remotely we need to create an ssh tunnel through to the swarm cluster to access our service, exposing the port 8080 on your local machine.

We have the ip address of the swarm master node which we obtained from docker-machine:

In [ ]:
master1ip = m.ip(machine='swmaster1')

Then open your web browser at the page http://<master1ip>:8080 (replace <master1ip>) and you should see a lovely blue whale, as below:

![](images/docker.png)


Alternatively we can obtain a textual representation on the command-line:

In [ ]:
!wget -O - http://$master1ip:3030

In [ ]:
#!wget -O - http://192.168.99.102:3030

# Scaling the service

Now we can scale the service to 3 replicas:

From the command line using:
#### docker $(docker-machine config swmaster1) service scale docker-demo=3

or using the update method of the service object:

In [ ]:
service.update??

In [ ]:
service_mode = docker.types.ServiceMode('replicated', 3)
print("Update service using:" + str(service_mode))

service.update(name=service.name, mode=service_mode, endpoint_spec=endpoint_spec, command="/bin/docker-demo")
#service.update(name=service.name, mode=service_mode, command="sleep 600")

In [ ]:
#service.remove()

In [ ]:
service.attrs

In [ ]:
print("tasks(containers) running on node swmaster1:" + str(swmaster1.containers.list()) )
print("tasks(containers) running on node swnode1:" + str(swnode1.containers.list()) )
print("tasks(containers) running on node swnode2:" + str(swnode2.containers.list()) )

# Rolling update

Now we will see how we can perform a rolling update.

We initially deployed version 20 of the service, now we will upgrade our whole cluster to version 20 


In [ ]:
!docker $(docker-machine config swmaster1) service ps docker-demo

In [ ]:
!docker $(docker-machine config swmaster1) service ps docker-demo

In [ ]:
!docker $(docker-machine config swmaster1) service update --detach=true --image mjbright/docker-demo:21 docker-demo

In [ ]:
#!docker $(docker-machine config swmaster1) service update --detach=false --image mjbright/docker-demo:21 docker-demo

In [ ]:
#image="mjbright/docker-demo:21"

####container_spec = docker.types.ContainerSpec(image, ['echo', 'hello'])
        
####task_tmpl = docker.types.TaskTemplate(container_spec)
 
#print("Update service using:" + str(image))

####service.update(name=service.name, task_template=task_tmpl, command="sleep 600")
#service.update(name=service.name, image=image)

In [ ]:
!docker $(docker-machine config swmaster1) service ps docker-demo

In [ ]:
!docker $(docker-machine config swmaster1) service ps docker-demo

### Verifying the service has been updated

Then open your web browser at the page http://localhost:8080 and you should now see a lovely **red** whale.


![](images/docker_red.png)



In [ ]:
!wget -O - http://$master1ip:3030

# Leaving the Swarm Cluster

In [ ]:
show_nodes(swmaster1)

In [ ]:
swmaster1.nodes.list()

In [ ]:
for node in swmaster1.nodes.list():
    #print(node)
    hostname=node.attrs['Description']['Hostname']
    print(str(node) + " ==> " + hostname + " Spec: " + str(node.attrs['Spec']))
    #pp.pprint(node.attrs)

In [ ]:
for node in swmaster1.nodes.list():
    #print(node)
    hostname=node.attrs['Description']['Hostname']
    print(str(node) + " ==> " + hostname)
    pp.pprint(node.attrs)

In [ ]:
swmaster1.nodes.list()

In [ ]:
swnode1.swarm.leave()
swnode2.swarm.leave()

## TODO: remove nodes

In [ ]:
node=swmaster1.nodes.get("ya6bpopa0m")

In [ ]:
node.attrs

In [ ]:
node.remove?

In [ ]:
for node in swmaster1.nodes.list():
    hostname=node.attrs['Description']['Hostname']
    print(str(node) + " ==> " + hostname)
    #if hostname == 'swnode2':
    #    #ret=swnode2.swarm.leave()
    #    print(ret)
    #if hostname == 'swnode1':
        #ret=swnode1.swarm.leave()
        #swmaster1.swarm.
        #ret=swmaster1.swarm.leave(id=node.id)
        #print(ret)

In [ ]:
swmaster1.swarm.leave(force=True)

In [ ]:
#swnode2.swarm.leave(id=swnode2.node_id)

In [ ]:
#swnode2.swarm.leave??

In [ ]:
for node in swmaster1.nodes.list():
    pp.pprint(node.attrs['Status'])

# TODO:  drain a node

We can drain a node effectively placing it in 'maintenance mode'.

Draining a node means that it no longer has running tasks on it.

In [ ]:
docker $(docker-machine config swmaster) node ls

Let's drain swnode1

In [ ]:
!docker $(docker-machine config swmaster1) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) node update --availability drain swnode1

and now we see that all services on swnode1 are shutdown

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

# TODO:  remove a service

Now let's cleanup by removing our service

In [ ]:
docker $(docker-machine config swmaster) service rm docker-demo

We can check that the service is no longer running:

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) ps

# Deploying a stack

**NOTE**: The stack concept exists in the docker client but not yet in the API, so we cannot do this from Python

In [ ]:
cd /home/mjb/z/bin/DEMOS_TUTORIALS/2017-May-PyconUS/docker-cli/alexellis.faas

In [ ]:
!docker swarm init --advertise-addr 10.9.0.10

In [ ]:
!grep -a REQUEST /tmp/dump.txt | tail -6

In [ ]:
!docker stack deploy func --compose-file docker-compose.yml

We can see that there is no "stack" operation in the API requests, but many lower level operations initiated by the Docker client

In [ ]:
!grep -a REQUEST /tmp/dump.txt

Listing services we see that may are marked as 0/1 REPLICAS.

This means that they have not yet been instanciated - probably we are still waiting for the appropriate images to be downloaded

In [ ]:
!docker service ls

Eventually all services are available as here

In [ ]:
!docker service ls

In [ ]:
!docker service logs func_gateway

In [ ]:
!docker stack ls

In [ ]:
!docker stack rm func

In [ ]:
!grep -a REQUEST /tmp/dump.txt

# TODO: working with networks

# TODO: working with volumes

<a name="OTHERPY" />
# Other Python  modules for Docker
<a href="#TOP"> TOP </a>



<a name="ANSIBLE" />
# Ansible Container
<a href="#TOP"> TOP </a>